In [ ]:
import json
import xnetwork as xnet
from tqdm.auto import tqdm
from collections import defaultdicts
from multiprocessing import Pool

In [ ]:
import glob
from functools import partial
from multiprocessing import Poo
def get_papers_year(valid_uids, filename):
    papers_by_year = dict()
    with open(filename) as file:
        for line in file:
            paper = json.loads(line)
            referencesData = wos.utilities.getReferences(paper)
            ref_uids = wos.utilities.getReferencesUIDs(referencesData)
            for ref in ref_uids:
                if ref in valid_uids:
                    publicationInfo = wos.utilities.getPublicationInfo(paper)
                    year = wos.utilities.getPublicationYear(publicationInfo)
                    year = int(year)
                    if ref in papers_by_year:
                        if year in papers_by_year[ref]:
                            papers_by_year[ref][year] += 1
                        else:
                            papers_by_year[ref][year] = 1
                    else:
                        papers_by_year[ref] = dict()
                        papers_by_year[ref][year] = 1
    return papers_by_year


def get_papers_year_union(valid_uids):
    print(files)
    
    pool = Pool(6)
    result = pool.map(partial(get_papers_year, valid_uids), files)
    
    print('join result...')
    join_result = result[0]
    for r in result[1:]:
        for paper, cit_by_year in r.items():
            for y,c in cit_by_year.items():
                if paper in join_result:
                    if y in join_result[paper]:
                        join_result[paper][y] += c
                    else:
                        join_result[paper][y] = c
                else:
                    join_result[paper] = dict()
                    join_result[paper][y] = c
    
    return join_result


print('get_impact...')
chu_journal = 'adv. funct. mater.'
file = '%s_completedata_110423_bardo_infomap_110423_voting_schema_top15xnet' % chu_journal
net = xnet.xnet2igraph(file)

papers_per_year = defaultdict(lambda:defaultdict(lambda:0))
for vtx in net.vs:
    papers_per_year[vtx['cluster_top15']][vtx['year']] += 1

valid_uids = net.vs['wos_id']
valid_uids = set(valid_uids)

dyear = 1

papers_by_year = get_papers_year_union(valid_uids)

In [ ]:
def get_impact(chu_journal): # citations
    
    print('get_impact...')
    
    file = '%s_completedata_110423_bardo_infomap_110423_voting_schema.xnet' % chu_journal
    net = xnet.xnet2igraph(file)

    papers_per_year = defaultdict(lambda:0)
    for vtx in net.vs:
        papers_per_year[vtx['year']] += 1
    
    valid_uids = net.vs['wos_id']
    valid_uids = set(valid_uids)
    
    dyear = 1
    
    papers_by_year = get_papers_year_union(valid_uids)
    bardo_dist = defaultdict(lambda: 0.0)


    print('cits per year...')
    for y in range(2010, 2021): # 2000
        
        for paper in net.vs.select(year_le=y-1, year_ge=y-2):
            if paper['wos_id'] in papers_by_year:
                cits_by_year = papers_by_year[paper['wos_id']]

                if y in cits_by_year:
                    bardo_dist[y] += cits_by_year[y]
    
    print(bardo_dist.keys())
    out_copy = dict()
    C_others = defaultdict(lambda:0)
    P_others = defaultdict(lambda:0)
    for y, count_cit in bardo_dist.items():
        y1 = 0
        y2 = 0
        if y-1 in papers_per_year:
            y1 = papers_per_year[y-1]
        if y-2 in papers_per_year:
            y2 = papers_per_year[y-2]

        if y1 + y2 > 0:
            out_copy[y] = count_cit/(y1 + y2)
        else:
            out_copy[y] = 0

    json_str = json.dumps(out_copy)
    out = open('impact_factor_dist_journal_%s_details.json' % chu_journal, 'w')
    out.write(json_str)
    out.close()



In [ ]:
get_impact('Adv. Funct. Mater.')
get_impact('ACS Appl. Mater. Interfaces')